## 데이터 전처리

In [1]:
# 모듈 가져오기
import pandas as pd
from statsmodels.api import add_constant
from sklearn.model_selection import train_test_split
from statsmodels.api import OLS
from statsmodels.stats.outliers_influence import variance_inflation_factor
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.api import qqplot
import time
import itertools

In [2]:
# 데이터 가져오기 - 도요타 중고차 가격 데이터
Toyota = pd.read_csv('./Data/ToyotaCorolla.csv')

In [3]:
# Fuel_Type -> 더미화
dummies = pd.get_dummies(Toyota['Fuel_Type'])

In [4]:
# 불필요한 변수 제거 및 더비변수 추가
remove_cols = ['Id', 'Model', 'Fuel_Type']
tmp = Toyota.drop(remove_cols, axis = 1)

In [5]:
# 데이터 결합
toyota_new = pd.concat([tmp, dummies], axis = 1)

In [6]:
# 회귀 모델링을 위한 상수 추가 - add_constant
df_reg = add_constant(toyota_new, has_constant = 'add')

In [7]:
# 데이터 분할
X = df_reg.drop('Price', axis = 1)
y = df_reg['Price']
X_train, X_test, y_train, y_test = train_test_split(X,y,train_size = 0.7, random_state = 42)

## 회귀를 위한 변수 선택
### 1. 통계적으로 유의하지 않은 변수 제거
### 2. 다중공선성을 가지는 변수 제거(VIF > 5)
### 3. AIC값이 가장 작은 모델을 선택

In [8]:
# 함수 정의 - 통계적으로 유의하지 않은 변수 제거
def pvalue_process(X_train, X_test, y_train):
    # 훈련데이터 학습
    full_model = OLS(y_train, X_train)
    full_model_fit = full_model.fit()

    # 통계적으로 유의미하지 않은 변수 제거
    parameters = full_model_fit.params.index
    pvalues = full_model_fit.pvalues
    pvalue_max = pvalues.max()
    while pvalue_max >= 0.05:
        parameters = full_model_fit.params.index
        pvalues = full_model_fit.pvalues
        pvalue_max = pvalues.max()
        for idx, pval in enumerate(pvalues):
            if pval == pvalue_max:
                remove_cols.append(parameters[idx])
                # 제거된 변수를 제외한 나머지 변수들로 새롭게 데이터 정의
                X_train = X_train.drop(remove_cols[-1], axis = 1)
                X_test = X_test.drop(remove_cols[-1], axis = 1)
                full_model = OLS(y_train, X_train)
                full_model_fit = full_model.fit()
    else:
        pass
    return full_model_fit, X_train, X_test

In [9]:
# 다중공선성 확인 - VIF값이 5보다 큰 변수들 제거
def multicolinearity(X_train, X_test):
    models = list()
    VIF = [variance_inflation_factor(X_train.values, i) for i in range(X_train.shape[1])]
    VIF_max = np.array(VIF).max()
    while VIF_max > 5:
        VIF = [variance_inflation_factor(X_train.values, i) for i in range(X_train.shape[1])]
        VIF_max = np.array(VIF).max()
        for idx, vif in enumerate(VIF):
            if vif == VIF_max:
                remove_cols.append(X_train.columns[idx])
                # 제거된 변수를 제외한 나머지 변수들로 새롭게 데이터 정의
                X_train = X_train.drop(remove_cols[-1], axis = 1)
                X_test = X_test.drop(remove_cols[-1], axis = 1)
                full_model = OLS(y_train, X_train)
                full_model_fit = full_model.fit()
                
            else:
                pass
    return full_model_fit, VIF, X_train, X_test

In [10]:
# 1. 통계적으로 유의미하지 않은 변수 제거
full_model_fit_pvalue, X_train_new1, X_test_new1 = pvalue_process(X_train, X_test, y_train)

In [11]:
# 2. 다중공선성 확인하여 VIF값이 5보다 큰 변수들 제거
full_model_fit_vif, VIF, X_train_new2, X_test_new2 = multicolinearity(X_train_new1, X_test_new1)

In [12]:
# 2-1. 모델 결과값  확인
full_model_fit_vif.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                  Price   R-squared (uncentered):                   0.920
Model:                            OLS   Adj. R-squared (uncentered):              0.919
Method:                 Least Squares   F-statistic:                              952.4
Date:                Tue, 20 Oct 2020   Prob (F-statistic):                        0.00
Time:                        22:06:02   Log-Likelihood:                         -9536.7
No. Observations:                1005   AIC:                                  1.910e+04
Df Residuals:                     993   BIC:                                  1.916e+04
Df Model:                          12                                                  
Covariance Type:            nonrobust                                                  
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Mfg_Month          312.3143     26.316     11.868      0.000     260.673     363.955
Mfr_Guarantee     2195.1203    204.498     10.734      0.000    1793.822    2596.419
Guarantee_Period   662.1021     30.978     21.373      0.000     601.312     722.892
Airco             1531.2909    248.594      6.160      0.000    1043.462    2019.120
Automatic_airco   5288.3624    489.024     10.814      0.000    4328.722    6248.002
Boardcomputer     3435.3148    269.815     12.732      0.000    2905.841    3964.788
CD_Player         1287.6516    289.031      4.455      0.000     720.470    1854.833
Powered_Windows   2139.6879    238.924      8.956      0.000    1670.835    2608.541
Sport_Model       2297.0591    221.397     10.375      0.000    1862.600    2731.518
Tow_Bar           1043.0915    224.799      4.640      0.000     601.956    1484.227
CNG                -58.4797   1028.345     -0.057      0.955   -2076.458    1959.498
Diesel            2364.1535    320.063      7.387      0.000    1736.076    2992.231
==============================================================================
Omnibus:                       78.615   Durbin-Watson:                   1.930
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              259.548
Skew:                          -0.332   Prob(JB):                     4.36e-57
Kurtosis:                       5.399   Cond. No.                         76.1
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [13]:
# 2-2. CNG가 통계적으로 유의미하지 않다. -> 제거
full_model_fit_pvalue2, X_train_new3, X_test_new3 = pvalue_process(X_train_new2, X_test_new2, y_train)

In [14]:
# 2-3. 모델 결과값 확인
full_model_fit_pvalue2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                  Price   R-squared (uncentered):                   0.918
Model:                            OLS   Adj. R-squared (uncentered):              0.918
Method:                 Least Squares   F-statistic:                              1121.
Date:                Tue, 20 Oct 2020   Prob (F-statistic):                        0.00
Time:                        22:06:02   Log-Likelihood:                         -9546.6
No. Observations:                1005   AIC:                                  1.911e+04
Df Residuals:                     995   BIC:                                  1.916e+04
Df Model:                          10                                                  
Covariance Type:            nonrobust                                                  
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Mfg_Month          307.1608     26.492     11.594      0.000     255.174     359.148
Mfr_Guarantee     2278.2699    205.282     11.098      0.000    1875.434    2681.106
Guarantee_Period   668.1055     31.225     21.397      0.000     606.831     729.380
Airco             1600.6648    250.320      6.394      0.000    1109.450    2091.880
Automatic_airco   5494.4802    491.140     11.187      0.000    4530.691    6458.270
Boardcomputer     3965.7609    244.276     16.235      0.000    3486.405    4445.117
Powered_Windows   2198.4642    240.682      9.134      0.000    1726.162    2670.766
Sport_Model       2347.2772    222.986     10.527      0.000    1909.699    2784.855
Tow_Bar           1045.0300    226.046      4.623      0.000     601.449    1488.611
Diesel            2437.3725    322.255      7.563      0.000    1804.995    3069.750
==============================================================================
Omnibus:                       61.500   Durbin-Watson:                   1.934
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              185.460
Skew:                          -0.250   Prob(JB):                     5.34e-41
Kurtosis:                       5.044   Cond. No.                         36.4
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [15]:
# 2-4. 데이터 재정의
X_train = add_constant(X_train_new3)
X_test = add_constant(X_test_new3)

In [16]:
# 3. AIC가 가장 작은값을 가지는 모델을 선택
# 함수 정의 - 회귀 모델과, 모델의 AIC 값 추출
def processSubset(x, y, feature_set):
    model = OLS(y, x[feature_set])
    regression = model.fit()
    AIC = regression.aic    
    return {'model' : regression, "AIC" : AIC}

In [17]:
def getBest(x,y,k):
    tic = time.time()
    results = list()
    for combo in itertools.combinations(x.columns.difference(['const']), k):
        combo = list(combo) + ['const']
        results.append(processSubset(x,y,feature_set = combo))
    models = pd.DataFrame(results)
    bestModel = models.loc[models['AIC'].argmin()]
    toc = time.time()
    print('Processed', models.shape[0], 'models on', k, 'predictors in', (toc-tic), 'seconds')
    return models, bestModel

In [28]:
# 3-1. 최적 모델 확인
models = getBest(X_train, y_train, len(X_train.columns.difference(['const']))-1)[0]
best_model = getBest(X_train, y_train, len(X_train.columns.difference(['const']))-1)[1]
best_model['model'].summary()

Processed 10 models on 9 predictors in 0.03989410400390625 seconds
Processed 10 models on 9 predictors in 0.027907371520996094 seconds


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  Price   R-squared:                       0.647
Model:                            OLS   Adj. R-squared:                  0.644
Method:                 Least Squares   F-statistic:                     202.4
Date:                Tue, 20 Oct 2020   Prob (F-statistic):          7.86e-218
Time:                        22:10:41   Log-Likelihood:                -9127.0
No. Observations:                1005   AIC:                         1.827e+04
Df Residuals:                     995   BIC:                         1.832e+04
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Airco             1060.6378    165.385      6.413      0.000     736.095    1385.180
Automatic_airco   6025.7837    323.312     18.638      0.000    5391.332    6660.235
Boardcomputer     3387.3143    161.548     20.968      0.000    3070.301    3704.328
Diesel             547.4429    218.291      2.508      0.012     119.080     975.806
Guarantee_Period   246.4661     23.607     10.440      0.000     200.140     292.792
Mfr_Guarantee      594.8841    143.062      4.158      0.000     314.147     875.621
Powered_Windows    778.5676    163.283      4.768      0.000     458.148    1098.987
Sport_Model        730.2105    152.985      4.773      0.000     430.000    1030.421
Tow_Bar           -435.0761    154.435     -2.817      0.005    -738.132    -132.020
const             7094.0214    176.906     40.101      0.000    6746.870    7441.173
==============================================================================
Omnibus:                      224.329   Durbin-Watson:                   2.122
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              786.842
Skew:                           1.049   Prob(JB):                    1.38e-171
Kurtosis:                       6.793   Cond. No.                         24.1
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## 전진선택법

In [31]:
# 변수 사전 정의
predictors = X_train.columns

In [47]:
# 전진 선택법
def forward(x,y,predictors):
    remainingPredictors = [p for p in X_train.columns.difference(['const']) if p not in predictors]
    tic = time.time()
    results = list()
    for p in remainingPredictors:
        results.append(processSubset(X_train, y_train, feature_set=predictors + [p] + ['const']))
        models = pd.DataFrame(results)
        bestModel = models.loc[models['AIC'].argmin(), :]
        toc = time.time()
        print('Processed', models.shape[0], 'models on', len(predictors) + 1, 'predictors in', (toc-tic))
        print('Selected predictors:', bestModel['model'].model.exog_names, 'AIC:' , bestModel[0])
        return bestModel

In [48]:
# 전진 선택법 모델
def forward_model(x,y):
    fModels = pd.DataFrame(columns = ['AIC', 'model'])
    tic = time.time()
    predictors = list()
    for i in range(1, len(x.columns.difference(['const']))+1):
        forwardResult = forward(x,y,predictors)
        if i > 1:
            if forwardResult['AIC'] > fmodelBefore:
                break
        fModels.loc[i] = forwardResult
        predictors = fModels.loc[i]['model'].model.exog_names
        fmodelBefore = fModels.loc[i]['AIC']
        predictors = [k for k in predictors if k != 'const']
    toc = time.time()
    print("Total elapesed time : ", (toc - tic), "seconds.")
    return (fModels['model'][len(fModels['model'])])

In [50]:
forward_model(X_train, y_train).summary()

Processed 1 models on 1 predictors in 0.004977226257324219
Selected predictors: ['Airco', 'const'] AIC: <statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x000001E3745C8D00>
Processed 1 models on 2 predictors in 0.00498509407043457
Selected predictors: ['Airco', 'Automatic_airco', 'const'] AIC: <statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x000001E374438160>
Processed 1 models on 3 predictors in 0.0049855709075927734
Selected predictors: ['Airco', 'Automatic_airco', 'Boardcomputer', 'const'] AIC: <statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x000001E374438850>
Processed 1 models on 4 predictors in 0.003987789154052734
Selected predictors: ['Airco', 'Automatic_airco', 'Boardcomputer', 'Diesel', 'const'] AIC: <statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x000001E374438A60>
Total elapesed time :  0.055847883224487305 seconds.


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  Price   R-squared:                       0.589
Model:                            OLS   Adj. R-squared:                  0.588
Method:                 Least Squares   F-statistic:                     478.0
Date:                Tue, 20 Oct 2020   Prob (F-statistic):          1.14e-192
Time:                        22:36:47   Log-Likelihood:                -9203.2
No. Observations:                1005   AIC:                         1.841e+04
Df Residuals:                    1001   BIC:                         1.843e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Airco            1477.4570    154.628      9.555      0.000    1174.025    1780.889
Automatic_airco  6371.4794    338.500     18.823      0.000    5707.229    7035.730
Boardcomputer    3480.3194    170.422     20.422      0.000    3145.894    3814.744
const            8624.9574    105.720     81.583      0.000    8417.498    8832.417
==============================================================================
Omnibus:                      287.676   Durbin-Watson:                   2.097
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1288.433
Skew:                           1.268   Prob(JB):                    1.66e-280
Kurtosis:                       7.933   Cond. No.                         5.64
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## 후진제거법

In [55]:
# 후진 제거법
def backward(x,y,predictors):
    tic = time.time()
    results = list()
    for combo in itertools.combinations(predictors, len(predictors)-1):
        results.append(processSubset(x,y, list(combo) + ['const']))
    models = pd.DataFrame(results)
    bestModel = models.loc[models['AIC'].argmin(), :]
    toc = time.time()
    print("Processed",models.shape[0],"models on",len(predictors)-1,
          "predictors in",(toc - tic))
    print("Selected predictors :",bestModel['model'].model.exog_names,
          ' AIC:',bestModel[0])
    return bestModel

In [56]:
# 후진 제거법 모델
def backward_model(x,y):
    BModels = pd.DataFrame(columns = ['AIC', 'model'])
    tic = time.time()
    predictors = x.columns.difference(['const'])
    BmodelBefore = processSubset(x,y,predictors)['AIC']
    while (len(predictors)>1):
        backwardResult = backward(x,y,predictors)
        if backwardResult['AIC'] > BmodelBefore:
            break
        BModels.loc[len(predictors)-1] = backwardResult
        predictors = BModels.loc[len(predictors)-1]['model'].model.exog_names
        BmodelBefore = backwardResult['AIC']
        predictors = [k for k in predictors if k != 'const']
    
    
    toc = time.time()
    print("Total elapsed time :",(toc - tic), "seconds.")
    return BModels['model'].dropna().iloc[0]

In [58]:
backward_model(X_train,y_train).summary()

Processed 10 models on 9 predictors in 0.04587721824645996
Selected predictors : ['Airco', 'Automatic_airco', 'Boardcomputer', 'Diesel', 'Guarantee_Period', 'Mfr_Guarantee', 'Powered_Windows', 'Sport_Model', 'Tow_Bar', 'const']  AIC: <statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x000001E374A15790>
Processed 9 models on 8 predictors in 0.023931264877319336
Selected predictors : ['Airco', 'Automatic_airco', 'Boardcomputer', 'Guarantee_Period', 'Mfr_Guarantee', 'Powered_Windows', 'Sport_Model', 'Tow_Bar', 'const']  AIC: <statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x000001E374A15F70>
Total elapsed time : 0.0857698917388916 seconds.


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  Price   R-squared:                       0.647
Model:                            OLS   Adj. R-squared:                  0.644
Method:                 Least Squares   F-statistic:                     202.4
Date:                Tue, 20 Oct 2020   Prob (F-statistic):          7.86e-218
Time:                        22:46:39   Log-Likelihood:                -9127.0
No. Observations:                1005   AIC:                         1.827e+04
Df Residuals:                     995   BIC:                         1.832e+04
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Airco             1060.6378    165.385      6.413      0.000     736.095    1385.180
Automatic_airco   6025.7837    323.312     18.638      0.000    5391.332    6660.235
Boardcomputer     3387.3143    161.548     20.968      0.000    3070.301    3704.328
Diesel             547.4429    218.291      2.508      0.012     119.080     975.806
Guarantee_Period   246.4661     23.607     10.440      0.000     200.140     292.792
Mfr_Guarantee      594.8841    143.062      4.158      0.000     314.147     875.621
Powered_Windows    778.5676    163.283      4.768      0.000     458.148    1098.987
Sport_Model        730.2105    152.985      4.773      0.000     430.000    1030.421
Tow_Bar           -435.0761    154.435     -2.817      0.005    -738.132    -132.020
const             7094.0214    176.906     40.101      0.000    6746.870    7441.173
==============================================================================
Omnibus:                      224.329   Durbin-Watson:                   2.122
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              786.842
Skew:                           1.049   Prob(JB):                    1.38e-171
Kurtosis:                       6.793   Cond. No.                         24.1
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## 단계적 방법

In [65]:
def Stepwise_model(x,y):
    stepModels = pd.DataFrame(columns = ['AIC', 'model'])
    tic = time.time()
    predictors = list()
    SmodelBefore = processSubset(x,y,predictors + ['const'])['AIC']
    for i in range(1, len(x.columns.difference(['const']))+1):
        forwardResult = forward(x,y,predictors)
        print('forward')
        stepModels.loc[i] = forwardResult
        predictors = stepModels.loc[i]['model'].model.exog_names
        predictors = [k for k in predictors if k != 'const']
        backwardResult = bacward(x,y,predictors)
        if backwardResult['AIC'] < forwardResult['AIC']:
            stepModels.loc[i] = backwardResult
            predictors = stepModels.loc[i]['model'].model.exog_names
            smodelBefore = stepModels.loc[i]['AIC']
            predictors = [k for k in predictors if k != 'const']
            print('backward')
        if stepModels.loc[i]['AIC'] > SmodelBefore:
            break
        else:
            smodelBefore = stepModels.loc[i]['AIC']
    toc = time.time()
    print("Total elapsed time : ", (toc - tic), "seconds")
    return stepModels['model'][len(stepModels['model'])]

In [66]:
Stepwise_model(X_train, y_train).summary()

Processed 1 models on 1 predictors in 0.004988908767700195
Selected predictors: ['Airco', 'const'] AIC: <statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x000001E374A212E0>
forward
Processed 1 models on 0 predictors in 0.005983114242553711
Selected predictors : ['const']  AIC: <statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x000001E3749EC310>
Processed 1 models on 2 predictors in 0.006975412368774414
Selected predictors: ['Airco', 'Automatic_airco', 'const'] AIC: <statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x000001E374A3C040>
forward
Processed 2 models on 1 predictors in 0.012940406799316406
Selected predictors : ['Automatic_airco', 'const']  AIC: <statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x000001E3749EC130>
Processed 1 models on 3 predictors in 0.003989458084106445
Selected predictors: ['Airco', 'Automatic_airco', 'Boardcomputer', 'const'] AIC: <statsmodels.regression.linear_model

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  Price   R-squared:                       0.589
Model:                            OLS   Adj. R-squared:                  0.588
Method:                 Least Squares   F-statistic:                     478.0
Date:                Tue, 20 Oct 2020   Prob (F-statistic):          1.14e-192
Time:                        23:02:28   Log-Likelihood:                -9203.2
No. Observations:                1005   AIC:                         1.841e+04
Df Residuals:                    1001   BIC:                         1.843e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Airco            1477.4570    154.628      9.555      0.000    1174.025    1780.889
Automatic_airco  6371.4794    338.500     18.823      0.000    5707.229    7035.730
Boardcomputer    3480.3194    170.422     20.422      0.000    3145.894    3814.744
const            8624.9574    105.720     81.583      0.000    8417.498    8832.417
==============================================================================
Omnibus:                      287.676   Durbin-Watson:                   2.097
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1288.433
Skew:                           1.268   Prob(JB):                    1.66e-280
Kurtosis:                       7.933   Cond. No.                         5.64
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""